In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

Starting path:/home/gregory_maruss/VESUVIUS_Challenge/jupyter notebooks
Current path:/home/gregory_maruss/VESUVIUS_Challenge


In [2]:
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.MONAI_Dict_Dataset_Module import MONAI_CSV_Scrolls_Dataset
import matplotlib.patches as patches
from lit_models.UNET_monai_lit import UNET_lit
from monai.visualize import matshow3d
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

2023-05-02 16:44:43,066 - Created a temporary directory at /tmp/tmpo5t1tj86
2023-05-02 16:44:43,068 - Writing /tmp/tmpo5t1tj86/_remote_module_non_scriptable.py
cuda


In [3]:
KAGGLE_DIR = PATH / "kaggle"

INPUT_DIR = KAGGLE_DIR / "input"

COMPETITION_DATA_DIR = INPUT_DIR / "vesuvius-challenge-ink-detection"

TRAIN_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_train_16_0.5.csv"
#TEST_DATA_CSV_PATH = COMPETITION_DATA_DIR / "data_test_1.csv"

In [4]:
dataset = MONAI_CSV_Scrolls_Dataset(
    z_dim=16,
    batch_size=1,
    data_csv_path=TRAIN_DATA_CSV_PATH,
    num_workers=8,
    num_samples=32,
    patch_size=(256,256),
    val_fragment_id=1,
    on_gpu=False,
    
    
)
#dataset.setup()

In [5]:
lit_model = UNET_lit(
        #wandb = True,
        z_dim = 16,
        patch_size = (256,256),
        sw_batch_size=8 ,
        eta_min = 1e-7,
        t_max = 250,
        max_epochs = 1000,
        weight_decay =  1e-6,
        learning_rate = 0.00005,
        gamma = 0.85,)

FROM_CHECKPOINT = False
if FROM_CHECKPOINT:
    lit_model = lit_model.load_from_checkpoint('logs/Full_16_768to1024_batch/FocalDice_768epoch=168FBETA=0.54recall=0.45precision=0.57.ckpt', 
                                             #patch_size = (512,512),
                                             learning_rate = 0.0001,
                                              weight_decay =  .001,
                                              eta_min = 1e-7,
                                             sw_batch_size = 8)

wandb: Currently logged in as: gmarus. Use `wandb login --relogin` to force relogin


In [6]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="FBETA",
    mode="max",
    dirpath="logs/EFFWeightedBCE_Monai_Unet_half_16/",
    filename="WBCE_MonaiUnet512{epoch:02d}{val_loss:.2f}{FBETA:.2f}{recall:.2f}{precision:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='gpu',
        #benchmark=True,
        max_epochs=1000,
        check_val_every_n_epoch= 5,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/EFFWeightedBCE_Monai_Unet_half_16/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/EFFWeightedBCE_Monai_Unet_half_16/',
        #overfit_batches=1,
        precision=16,
        accumulate_grad_batches=1, 
        callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/FocalDice_768epoch=123FBETA=0.30recall=0.77precision=0.26.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Full_16_768to1024_batch/FocalDice_768epoch=168FBETA=0.54recall=0.45precision=0.57.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'

2023-05-02 16:44:51,131 - GPU available: True (cuda), used: True
2023-05-02 16:44:51,132 - TPU available: False, using: 0 TPU cores
2023-05-02 16:44:51,133 - IPU available: False, using: 0 IPUs
2023-05-02 16:44:51,134 - HPU available: False, using: 0 HPUs
# train: 2
# val: 1
2023-05-02 16:44:52,379 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Adjusting learning rate of group 0 to 5.0000e-05.
2023-05-02 16:44:52,391 - 
   | Name                 | Type                  | Params
----------------------------------------------------------------
0  | metrics              | ModuleDict            | 0     
1  | model                | FlexibleUNet          | 11.5 M
2  | loss                 | MaskedLoss            | 0     
3  | mine_focal           | FocalLoss             | 0     
4  | weighted_bce_loss    | BCEWithLogitsLoss     | 0     
5  | loss_dice            | DiceLoss              | 0     
6  | loss_tversky         | TverskyLoss           | 0     
7  | loss_bce             | SoftBCEWithLog

Sanity Checking: 0it [00:00, ?it/s]

/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:84: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 64. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:84: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


...

Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 3.8683e-05.
Adjusting learning rate of group 0 to 3.8944e-05.
Adjusting learning rate of group 0 to 3.9203e-05.
Adjusting learning rate of group 0 to 3.9460e-05.
Adjusting learning rate of group 0 to 3.9715e-05.


Validation: 0it [00:00, ?it/s]

Adjusting learning rate of group 0 to 3.9968e-05.
Adjusting learning rate of group 0 to 4.0218e-05.
Adjusting learning rate of group 0 to 4.0466e-05.
Adjusting learning rate of group 0 to 4.0711e-05.


/home/gregory_maruss/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [7]:
lit_model.lr

5e-05

In [8]:
#trainer.save_checkpoint("logs/224_smp_unet_32_.5_ver0.ckpt")

In [9]:
#torch.save(lit_model.state_dict(), 'kaggle/original_1kbuffer_1.pth')


In [10]:
torch.cuda.empty_cache()

In [11]:
data_module = MONAI_CSV_Scrolls_Dataset(
                z_dim=16,
                batch_size=1,
                data_csv_path=TEST_DATA_CSV_PATH,
                num_workers=8,
                num_samples=16,
                patch_size=(512,512),
                val_fragment_id=1,
                stage='predict',
                on_gpu =False,
                    )

data_module.setup(stage='predict')

NameError: name 'TEST_DATA_CSV_PATH' is not defined

In [ ]:
lit_model.lr

In [ ]:
lit_model.eval().to('cpu')
predictions = trainer.predict(lit_model.eval(), data_module)

In [ ]:
predictions

In [ ]:
def plot_image(image, title):
    fig = plt.figure()
    plt.title(title)
    plt.imshow(image, cmap="gray")
    
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    # pixels = (pixels >= thr).astype(int)
    
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import pandas as pd
import numpy as np
import PIL.Image as Image
import lovely_numpy as ln
submission_df = pd.read_csv("kaggle/input/vesuvius-challenge-ink-detection/sample_submission.csv")
test_df = pd.read_csv("kaggle/input/vesuvius-challenge-ink-detection/data_test_1.csv")

In [ ]:
test_df

In [ ]:
predictions_rle = []
THRESHOLD = .58
for mask_png_path, prediction in zip(test_df["mask_png"].values, predictions):
    prediction = prediction.numpy()
    mask = Image.open(mask_png_path)

    prediction_resized = np.array(Image.fromarray(prediction).resize(mask.size))
    
    prediction_masked = prediction_resized * mask
    
    prediction_thresholded = np.where(prediction_masked > THRESHOLD, 1, 0).astype(np.uint8)
    
    prediction_rle = rle(prediction_thresholded)

    predictions_rle.append(prediction_rle)

    if 1:
        #plot_image(prediction_resized, f"prediction resized {ln.lovely(prediction_resized)}")
        #plot_image(prediction_masked, f"prediction masked {ln.lovely(prediction_masked)}")
        plot_image(
            prediction_thresholded,
            f"prediction thresholded {ln.lovely(prediction_thresholded)}",
        )
    
submission_df["Predicted"] = predictions_rle
    
submission_df.to_csv("submission.csv", index=False)

submission_df

In [ ]:
#submission_df